# Interactive Plotting with Plotly

Xarray provides interactive plotting capabilities using [Plotly Express](https://plotly.com/python/plotly-express/) through the `.plotly` accessor on DataArrays. This enables creating interactive, zoomable, and hoverable plots directly from xarray data structures.

## Imports and Data

In [1]:
import xarray as xr

In [2]:
# Load tutorial data
airtemps: xr.Dataset = xr.tutorial.open_dataset("air_temperature")

# Convert to celsius
air = airtemps.air - 273.15
air.attrs = airtemps.air.attrs
air.attrs["units"] = "deg C"
air

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)> Size: 31MB
array([[[-31.95, -30.65, -29.65, ..., -40.35, -37.65, -34.55],
        [-29.35, -28.65, -28.45, ..., -40.35, -37.85, -33.85],
        [-23.15, -23.35, -24.26, ..., -39.95, -36.76, -31.45],
        ...,
        [ 23.45,  23.05,  23.25, ...,  22.25,  21.95,  21.55],
        [ 22.75,  23.05,  23.64, ...,  22.75,  22.75,  22.05],
        [ 23.14,  23.64,  23.95, ...,  23.75,  23.64,  23.45]],

       [[-31.05, -30.45, -30.05, ..., -41.15, -39.55, -37.35],
        [-29.55, -29.05, -28.95, ..., -42.15, -40.65, -37.45],
        [-19.95, -20.26, -21.05, ..., -42.35, -39.76, -34.65],
        ...,
        [ 23.25,  22.75,  23.05, ...,  22.25,  21.95,  21.64],
        [ 23.05,  23.55,  23.64, ...,  22.45,  22.35,  21.95],
        [ 23.14,  24.05,  24.25, ...,  23.25,  23.25,  23.45]],

       [[-30.85, -30.95, -30.85, ..., -38.85, -37.05, -34.45],
        [-28.55, -28.76, -29.15, ..., -42.85, -41.15, -37.45],
        [-16.95, -17.65, -18.95, ..., -41.95, -39.95, -34.95],
        ...,
...
        [ 21.64,  22.14,  24.34, ...,  22.34,  22.24,  21.54],
        [ 23.64,  24.74,  25.14, ...,  22.34,  22.34,  21.64],
        [ 25.04,  26.04,  25.64, ...,  22.94,  22.64,  22.64]],

       [[-27.36, -28.36, -29.66, ..., -29.86, -29.16, -28.36],
        [-23.26, -23.86, -24.66, ..., -31.86, -30.66, -28.86],
        [-10.76, -11.36, -11.86, ..., -32.66, -30.06, -26.26],
        ...,
        [ 20.54,  20.74,  22.24, ...,  21.94,  21.54,  21.14],
        [ 23.14,  24.04,  24.44, ...,  22.14,  21.94,  21.24],
        [ 24.64,  25.24,  25.34, ...,  22.54,  22.34,  22.04]],

       [[-28.06, -28.86, -29.86, ..., -31.46, -31.66, -31.36],
        [-23.26, -23.86, -24.76, ..., -33.56, -32.86, -31.46],
        [-10.16, -10.96, -11.76, ..., -33.26, -30.56, -26.86],
        ...,
        [ 20.64,  20.54,  21.94, ...,  22.14,  21.94,  21.54],
        [ 22.94,  23.74,  24.04, ...,  22.54,  22.54,  22.04],
        [ 24.54,  24.94,  24.94, ...,  23.34,  23.04,  22.54]]],
      shape=(2920, 25, 53))
Coordinates:
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
Attributes:
    long_name:     4xDaily Air temperature at sigma level 995
    units:         deg C
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NMC Reanalysis
    level_desc:    Surface
    statistic:     Individual Obs
    parent_stat:   Other
    actual_range:  [185.16 322.1 ]

## Line Plots

### Simple Line Plot

In [3]:
air1d = air.isel(lat=10, lon=10)
fig = air1d.plotly.line()
fig

### 2D Data with Automatic Assignment

With 2D data, dimensions are automatically assigned to slots.
The default slot order for line plots is: `x → color → facet_col → facet_row → animation_frame`

In [4]:
air2d: xr.DataArray = air.isel(lon=10, lat=[10, 15, 20])
# time → x, lat → color (automatic assignment)
fig = air2d.plotly.line()
fig

### Skipping Slots

Use `None` to skip a slot and let remaining dimensions fill later slots:

In [5]:
air3d = air.isel(lon=10, lat=[10, 15], time=slice(0, 100, 10))
# Skip color → time goes to x, lat to facet_col, nothing to color
fig = air3d.plotly.line(color=None)
fig

### Passing Plotly Arguments

Additional keyword arguments are passed directly to Plotly Express:

In [6]:
fig = air2d.plotly.line(title="Air Temperature Over Time")
fig

## Bar Charts

In [7]:
# Monthly mean temperature
monthly = air.isel(lat=10, lon=10).resample(time="ME").mean()
monthly = monthly.isel(time=slice(0, 12))
fig = monthly.plotly.bar()
fig

In [8]:
# With multiple dimensions
seasonal: xr.DataArray = air.isel(lon=10).groupby("time.season").mean()
seasonal_subset = seasonal.isel(lat=[10, 15, 20])
fig = seasonal_subset.plotly.bar()
fig

In [9]:
# With categorical colors
seasonal_subset = seasonal_subset.assign_coords(lat=seasonal_subset.lat.astype(str))
fig = seasonal_subset.plotly.bar()
fig

## Area Charts

In [10]:
fig = seasonal_subset.plotly.area()
fig

## Scatter Plots

By default, scatter plots show DataArray values on the y-axis:

In [11]:
air_sample = air.isel(time=slice(0, 50), lat=10, lon=10)
fig = air_sample.plotly.scatter()
fig

In [12]:
# With additional dimensions mapped to color
air_sample2d = air.isel(time=slice(0, 20), lat=[10, 15], lon=10)
# time → x, values → y, lat → color
fig = air_sample2d.plotly.scatter()
fig

### Dimension vs Dimension

To plot one dimension against another (instead of values on y-axis), explicitly set `y` to a dimension name:

In [13]:
# Plot lat vs lon coordinates, colored by temperature
air_snapshot = air.isel(time=0)
fig = air_snapshot.plotly.scatter(x="lon", y="lat", color="value")
fig

## Box Plots

In [22]:
fig = air.isel(lon=10, time=slice(0, 365)).plotly.box(x="lat")
fig

In [15]:
# Grouped by a dimension
air_box = air.isel(lon=[10, 15, 20], lat=[10, 15, 20], time=slice(0, 100))
fig = air_box.plotly.box(x="lat", color="lon")
fig

## Heatmaps with imshow

For 2D data, `imshow` creates heatmap visualizations:

In [16]:
air2d_spatial = air.isel(time=0)
fig = air2d_spatial.plotly.imshow()
fig

### Controlling Axis Orientation

The `x` and `y` parameters control which dimension appears on which axis by transposing the data:

In [17]:
# Swap axes: lon on x, lat on y
fig = air2d_spatial.plotly.imshow(x="lon", y="lat")
fig

### Faceted Heatmaps

Use `facet_col` to create small multiples:

In [18]:
air_monthly = air.resample(time="ME").mean().isel(time=slice(0, 4))
fig = air_monthly.plotly.imshow(x="lon", y="lat", facet_col="time")
fig

### Animated Heatmaps

Use `animation_frame` to create animated plots:

In [19]:
air_anim = air.isel(time=slice(0, 20, 2))
fig = air_anim.plotly.imshow(x="lon", y="lat", animation_frame="time")
fig

## Customizing Plots

All methods return a `plotly.graph_objects.Figure` that can be further customized using Plotly's API:

In [20]:
fig = air2d.plotly.line()
fig.update_layout(
    title="Customized Air Temperature Plot",
    xaxis_title="Date",
    yaxis_title="Temperature (°C)",
    template="plotly_white",
)
fig

## Dimension-to-Slot Assignment

The `.plotly` accessor uses automatic dimension-to-slot assignment.
Dimensions fill slots in order based on the plot type:

| Method | Slot Order |
|--------|------------|
| `line` | x → color → facet_col → facet_row → animation_frame |
| `bar` | x → color → facet_col → facet_row → animation_frame |
| `area` | x → color → facet_col → facet_row → animation_frame |
| `scatter` | x → color → size → facet_col → facet_row → animation_frame (y="value" default) |
| `box` | x → color → facet_col → facet_row → animation_frame |
| `imshow` | x → y → facet_col → animation_frame |

### Assignment Rules

1. **Explicit assignments** lock a dimension to a slot
2. **None** skips a slot entirely
3. **Remaining dimensions** fill remaining slots by position
4. **Error** if dimensions remain after all slots are filled